In [1]:
import json
import re
import pandas as pd
# from mgvllm.logics.chat import send_text
import numpy as np
from sentence_transformers import  SentenceTransformer,util
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
from collections import Counter

In [4]:
import sys
sys.path

['C:\\Users\\10634',
 'D:\\anaconda\\python311.zip',
 'D:\\anaconda\\DLLs',
 'D:\\anaconda\\Lib',
 'D:\\anaconda',
 '',
 'C:\\Users\\10634\\AppData\\Roaming\\Python\\Python311\\site-packages',
 'D:\\anaconda\\Lib\\site-packages',
 'D:\\anaconda\\Lib\\site-packages\\win32',
 'D:\\anaconda\\Lib\\site-packages\\win32\\lib',
 'D:\\anaconda\\Lib\\site-packages\\Pythonwin']

In [2]:
path = r'C:\Users\10634\Desktop\重聚类\test_data.json'
with open(path,'r',encoding='utf-8') as f:
    data = json.load(f)

In [8]:
s = SentenceTransformer('shibing624/text2vec-base-chinese')
recluster_q = []
for q in data['key'].keys():
    if q != '其他':
        limit = 15
        if len(data['key'][q]['quindex_list']) > limit:
            recluster_q.append(q)
pre_dic = {}
for r_q in recluster_q:
    print(f'开始重聚类\t{r_q}')
    qlist = list(data['key'][r_q]['question'].keys())
    alen = len(qlist)
    quindex_list = list(data['key'][r_q]['quindex_list'])
    q_number = 5
    ncluster = int(alen / q_number)
    raw_vector = s.encode(qlist)
    model = AgglomerativeClustering(n_clusters=ncluster)
    model.fit(raw_vector)
    df = pd.concat([pd.Series(quindex_list),pd.Series(qlist),pd.Series(model.labels_)],axis=1)
    df.columns = ['quindex','question','label']
    c = Counter(df['label'])
    c = dict(c)
    label = list(c.keys())
    rdic = {}
    for i in label:
        quindex_list = list(df[df['label'] == i]['quindex'])
        dic = {}
        lst = list(df[df['label'] == i]['question']) #在原始类别下新聚类别对应的问题列表
        dic['percentage'] = len(lst) / alen # 计算占比
        dic['quindex'] = quindex_list
        dic['question'] = lst
#         print(df)
#         print(lst)
        qu_vector = s.encode(lst)     # 计算新聚类别下所有问题的均值向量
        center_mean = np.mean(qu_vector,axis=0)
        dic['mean_vector'] = center_mean
        rdic[f'{i}'] = dic


    mean_list = [rdic[i]['mean_vector'] for i in rdic.keys()]
    mean_list = np.array(mean_list)
    qulist = [item for i in rdic.keys() for item in rdic[i]['question']] # 根据标签分好的原始类别对应的所有问题列表
    quindex = [item for i in rdic.keys() for item in rdic[i]['quindex']]

    all_qu_vector = s.encode(qulist)
    all_sim = np.array(util.pytorch_cos_sim(all_qu_vector,mean_list))
    # all_sim = np.transpose(all_sim) # 给相似度矩阵转置，方便索引
    df_sim = pd.DataFrame(all_sim)
    df_sim.index,df_sim.columns = qulist, list(rdic.keys())
    ind_list = []
    for i in df_sim.index:
        max_sim = max(list(df_sim.loc[i]))
        ind_tuple = np.where(np.array(df_sim) == max_sim)
        ind_list.append([int(ind_tuple[0]),int(ind_tuple[1])])

    rdic1 = {}
    for i in ind_list:
        if str(i[1]) not in rdic1:
            rdic1[str(i[1])] = [qulist[i[0]]]
        else:
            rdic1[str(i[1])].append(qulist[i[0]])
    rdic_later = {}
    for keys,values in rdic1.items():
        dic = {}
        per = len(values) / alen
        dic['percentage'] = per
        dic['question'] = values
        rdic_later[keys] = dic
    pre_dic[r_q] = rdic_later

开始重聚类	如何高效管理？
开始重聚类	那在什么地方？
开始重聚类	哎我上次消费个一笔钱我就分期行不行啊？
开始重聚类	费用多少？
开始重聚类	问一下就是说？
开始重聚类	到时候再联系好不好？
开始重聚类	都是正常的吗？
开始重聚类	我这张卡额度多少？
开始重聚类	头发的来源及相关问题
开始重聚类	要多要多久？
开始重聚类	头发问题，植发是否可行？
开始重聚类	我就想问的你们怎么收费的？
开始重聚类	哪个都可以？
开始重聚类	我装你能不能认识我？
开始重聚类	电话归属地不明，请确认


In [3]:
with open(r'C:/Users/10634/Desktop/重聚类/test_data_gpt.json',encoding='utf-8') as f:
    data1 = json.load(f)
data1

{'如何高效管理？': {'0': {'percentage': 0.05813953488372093,
   'question': ['里面你是指对学员还是对你们的？',
    '啊哈不需要你怎么说你说。',
    '云云部署现在呵呵。现在怎么说呢？',
    '你哪个公司，你就说哪个公司就。',
    '是不是不止一个人在不阶得，那就是在你们的CRM目里，我们是不给你党就是说。'],
   'new_center': '无法总结'},
  '1': {'percentage': 0.11627906976744186,
   'question': ['但是一八年之前，你们原来那个平台大概有多少门课程？',
    '数据大屏就是可以实时看到我们整个的一个平台的一个大致的一个情况，比如说课程的这些，受课受欢迎的一些课程是什么，然后课程的一些占比，然后学习活动参与情况，员工的活跃情况，然后考试积分等等，然后还可以按时间和部门去筛选。',
    '想想想看一看有没有大的偏差，如果没有大的偏差，那我们做一些，做一些背景，做一些这种的调整，其实还比较好改。',
    '种风格的，然后背景，然后我们把它改过去，然后反正就是今天的方案是这样，然后就是几个老师大概看一下我们的主要的思路，然后看思路上需不需要调整，那其实从整个背景，包括图片然后这些其实都是比较好，比较好改的。',
    '这个有没有配，他就是比如说学员进来之后，他看到的是八个模块，我们比如说配了八个模块，如果讲师进来之后，他其实会有另外一个多一个模块，就是他讲关于他的一些功能的一些权限。',
    '他这里有一个可以手动积分的，就可以你直接把你比如这个班的一些人员，他最后获得多少积分，直接可以导导进去，导到系统里面去。',
    '这个我们可以到时候一块看下，具体是哪个业务角色的人要用，核心要用哪些功能，可以一块评估下，如果单嵌入无法满足的话，我们这边研究下整个嵌入的实现',
    '其实怎么说呢，就是他这个几个平台的话，都是各有流量。',
    '风险设置大家也是会自己调整的，调整之后我们会及时提醒，比如说哪些客户信息会有跟进，或者是近期客户提出了很多的价值依据，客户依据这样。',
    '对那个语音模型需要淑芬同学去配，或者是怎么样，然后如果说它只需要关键词的鉴别的话，就是只配成

[[0, 0],
 [1, 0],
 [2, 0],
 [3, 0],
 [4, 1],
 [5, 0],
 [6, 0],
 [7, 2],
 [8, 0],
 [9, 0],
 [10, 0],
 [11, 0],
 [12, 1],
 [13, 2],
 [14, 0],
 [15, 2],
 [16, 0],
 [17, 2]]

In [113]:
df_sim['0']

你刚才给我打电话了是吗？                                                                                                                                                                                                                  0.884764
哪个打电话。                                                                                                                                                                                                                        0.822684
过来的时候打你这个电话吗。                                                                                                                                                                                                                 0.859166
问题是你不是这边的人，你手机你怎么跟我跟我打的。                                                                                                                                                                                                      0.833941
哪个在打电话？                                                                     

In [114]:
np.where(np.array(df_sim) == 0.8847637)

(array([0], dtype=int64), array([0], dtype=int64))

,0,2,1
你刚才给我打电话了是吗？,0.884764,0.692880,0.725541
哪个打电话。,0.822684,0.639368,0.669756
过来的时候打你这个电话吗。,0.859166,0.694577,0.762923
问题是你不是这边的人，你手机你怎么跟我跟我打的。,0.833941,0.809641,0.739547
哪个在打电话？,0.769113,0.989375,0.630854
你打我电话干嘛？,0.849238,0.626513,0.543732
我说你的电话怎么是武汉的？号码。,0.860744,0.578148,0.618150
我留什么电话吗？,0.724929,0.596640,0.875453
你怎么那个电话是武汉的？,0.816754,0.596582,0.566874
已经过去很久了，还在打电话吗？,0.788966,0.601288,0.686849
